Imports

In [ ]:
# !pip install pandas numpy
import pandas as pd
import numpy as np
from IPython.display import display

VARS

In [ ]:
# Load the CSV file
csv_file_path = 'input.csv'
df = pd.read_csv(csv_file_path)

# Replace NaN values with a placeholder for blanks only if you're sure they represent truly blank cells
BLANK_PLACEHOLDER = "<BLANK>"  # Define a constant for blank placeholders

# Iterate over all columns in the DataFrame
for column in df.columns:
    df[column].fillna(BLANK_PLACEHOLDER, inplace=True)

In [ ]:
# # df.head()
# # df.info()


# # Duplicates ----------
# print("Number of duplicate rows:", len(df[df.duplicated('account_url', keep=False)]))
# df[df.duplicated('account_url', keep=False)].sort_values(['account_url'])

---

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display


from IPython.display import clear_output

def consolidate_rows(rows):
    """Consolidate rows with user input for conflict resolution, handling blank and non-blank values appropriately."""
    consolidated = rows.iloc[0].copy()  # Start with the first row as the base for consolidation
    num_merge_conflicts = sum(len(rows[column].dropna().unique()) > 1 for column in rows.columns)
    print(f"There are {num_merge_conflicts} merge conflicts to resolve: ")
    
    for column in rows.columns:
        # Dropping NaN to consider non-empty values for conflict detection
        non_empty_values = rows[column].dropna().unique()
        
        if len(non_empty_values) == 1:
            consolidated[column] = non_empty_values[0]
        elif len(non_empty_values) > 1:
            print(f"\nConflict in '{column}' column with unique values: {non_empty_values}")
            conflict_df = rows[[column]].dropna().drop_duplicates().reset_index(drop=True)
            display_df = conflict_df.reset_index().rename(columns={"index": "Choice"})
            display_df['Choice'] += 1  # Adjust choice numbering to start from 1
            display(display_df)
            
            chosen_index = input(f"Select the row number to keep for {column} (1-{len(conflict_df)}), or type 'exit' to stop: ")
            clear_output()  # Clear the output after receiving input
            if chosen_index.lower() == 'exit':
                print("Operation stopped by user. Saving progress...")
                return None
            
            try:
                chosen_index = int(chosen_index) - 1
                if 0 <= chosen_index < len(conflict_df):
                    consolidated[column] = conflict_df.iloc[chosen_index, 0]
                else:
                    print("Invalid row number. No changes made to this column.")
            except ValueError:
                print("Invalid input. Please enter a valid row number or 'exit'.")
                return None
        else:
            consolidated[column] = np.nan
    return consolidated



def has_conflict(group):
    """Check if there is any conflict within the group, ignoring NaN values."""
    return any(len(group[column].dropna().unique()) > 1 for column in group.columns)

def main(csv_file_path):
    df = pd.read_csv(csv_file_path, dtype=str)
    original_order_df = df.copy()

    duplicates = df[df.duplicated('account_url', keep=False)]
    
    if duplicates.empty:
        print("No duplicates found.")
        df.to_csv('result.csv', index=False)
        return df
    
    consolidated_list = []
    total_conflicts = 0  # Initialize total conflicts count
    for account_url, group in duplicates.groupby('account_url'):
        if not has_conflict(group):
            continue
        
        total_conflicts += sum(len(group[column].dropna().unique()) > 1 for column in group.columns)
        
    print(f"There are {total_conflicts} merge conflicts to resolve: ")
    
    for account_url, group in duplicates.groupby('account_url'):
        if not has_conflict(group):
            continue
        
        print(f"\nProcessing next duplicate group for account_url: {account_url}")
        display(group)
        consolidated_row = consolidate_rows(group)
        if consolidated_row is None:
            print("Operation stopped by user. Saving progress...")
            break
        consolidated_list.append(consolidated_row)
    
    if consolidated_list:
        consolidated_df = pd.DataFrame(consolidated_list, columns=df.columns)
        df = pd.concat([original_order_df, consolidated_df], ignore_index=False)
        df = df.drop_duplicates('account_url', keep='last')
    
    df.sort_index(inplace=True)
    df.to_csv('result.csv', index=False)
    print("Consolidation complete. Results saved to 'result.csv'.")
    return df

csv_file_path = 'input.csv'
df_processed = main(csv_file_path)
